## Kaggle Survey 소개
- Kaggle이라는 Data Science Community가 있습니다. Kaggle에서는 매년 Kaggle 사용자들을 중심으로 설문조사를 시행합니다. 해당 설문조사 결과는 전세계에 뛰어난 Data Scientist들이 어떻게 일하고 어떤 것들을 공부하고 있는지 알 수 있는 좋은 자료입니다.

- 이번에 사용하는 데이터셋은 2021 Kaggle Machine Learning Survey 데이터셋을 사용 활동하는 Kaggler들에 대해 살펴보려고 합니다.

### 0. 데이터 소개
 
- 데이터의 첫번째 Row에는 각 질문이 어떤 것이었는지 대한 내용이 적혀 있습니다.

- 해당 데이터가 어떻게 수집되었는지, 활용하려면 어떻게 해야하는지 관심 있는 분들은 해당 데이터셋을 다운로드 하면 supplementary_data 폴더에서 해당 내용을 활용할 수 있습니다.

- 아래 링크에서 데이터를 다운로드 받아봅시다. 해당 데이터셋을 받기 위해서는 Kaggle에 회원가입이 되어있어야 하며, 대회 규정에 대해 동의하여야 합니다.

Source : https://www.kaggle.com/c/kaggle-survey-2021/data

### 1. 라이브러리 및 데이터 불러오기

In [ ]:
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_rows = 100

import warnings
warnings.filterwarnings('ignore')

In [ ]:
survey = pd.read_csv("../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv")
survey

### 2. 데이터 전처리

In [ ]:
# 불러온 dataframe의 정보를 요약 정보를 확인합니다.
survey.info()

In [ ]:
## 함수를 작성해보세요.
# 앞에서부터 column 7개 자르기
# 1. iloc
#survey.iloc[:, :7].info()

# 2. columns를 활용
survey[survey.columns[:7]].info()

In [ ]:
# 해당 데이터셋은 column이 너무 많아 전체적인 정보를 확인하기 힘듭니다.


### 3. 한국사람 찾기

In [ ]:
# 잘라온 데이터에서 한국은 어떤 텍스트로 표현이 되는지 찾아보세요.
survey[survey.Q3.str.contains("Korea")].Q3.unique() ## -> "South Korea"

In [ ]:
# 한국 사람들을 불러와 봅시다
korean = survey.loc[survey.Q3 == "South Korea"]
korean

#### 성별에 대한 분석

In [ ]:
# 성별에 해당하는 column의 countplot을 그려보세요.
sns.countplot(data=korean, x="Q2")
korean.Q2.value_counts()

# 성별에 따른 응답시간 차이
# 1. 응답 시간 column을 숫자(int) 타입으로 변경.
korean["Time from Start to Finish (seconds)"] = korean["Time from Start to Finish (seconds)"].astype(int)
# 2. pivot table 계산
pd.pivot_table(data=korean, index="Q2", values="Time from Start to Finish (seconds)")

#### 최종학력에 대한 분석

In [ ]:
# 최종학력에 해당하는 column의 countplot을 그려보세요.
plt.figure(figsize=(36, 6))
sns.countplot(data=korean, x="Q4", palette="Set2")
plt.show()

### 4. 각 나라별 학력 통계 계산하기

- pivot table을 이용하여 나라별 통계를 계산해봅시다.

#### pivot table 만들기

In [ ]:
countries = pd.pivot_table(data=survey, index="Q3", columns="Q4", aggfunc="count", fill_value=0).iloc[: ,:7]
countries

#### 특정 국가 가져오기

In [ ]:
# 미국과 캐나다를 가져와봅시다.
#countries.index
canada = countries.loc["Canada"]
display(canada) # 중간에 Series나 DataFrame이 있어도 출력 가능.
usa = countries.loc["United States of America"]
#canada + usa
display(usa)

### 5. 프로그래밍 언어 선호도 분석

- 어떤 column을 가져와야 할까요?

- Q7과 관련된 column들은 어떻게 다 가져올까요?

#### 프로그래밍 언어 선호도를 포함하는 column들을 불러오기

In [ ]:
# 1. 노가다
survey.iloc[: ,7:20]

# 2. column 이름이 Q7으로 시작하는 columns
q7_cols = [col for col in survey.columns if col.startswith("Q7")]
pls = survey[q7_cols]
pls

#### 언어별 정보가 합쳐진 DataFrame 만들기

In [ ]:
q7_rows = []

for i, row in pls.iterrows(): # 그냥 for문을 돌리면, column에 대해서 반복이 되기 때문에, row에 대해서 반복하기 위해서 iterrows를 사용.
    q7_rows.append(row[~row.isnull()].values) # Q7으로 시작하는 데이터에 대해서 각 row별로 nan이 아닌 데이터만 뽑는 작업.
    
survey["Q7"] = q7_rows # survey DataFrame에 추가.
survey = survey.drop(columns=q7_cols) # 기존에 Q7 다지선다 column 제거.
survey